In [1]:
import requests

## 브라우저인 척 하기위해 헤더 정보에 브라우저 정보 포함

def get_soup(url) :
  headers = {
      'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.0.0 Safari/537.36'
  }

  ## 해당 url로 요청 보내서 html 문서 받아오기
  r = requests.get(url, headers=headers)

  ## html 문서 가져오기
  html_doc = r.text

  ## BeautifulSoup 객체에 넘겨 파싱하기(정보를 태그 단위로 쪼개기)
  from bs4 import BeautifulSoup
  soup = BeautifulSoup(html_doc, 'html.parser')

  return soup

In [2]:
# 연도 페이지 링크


def get_all_box() :
  link1 = 'https://search.naver.com/search.naver?where=nexearch&sm=tab_etc&qvt=0&query=year%EB%85%84%20%EC%98%81%ED%99%94%20%EC%88%9C%EC%9C%84'

  query_idx = 3
  value_idx = 1

  years = range(2012, 2024)

  links = []
  for year in years :
    links.append(link1.replace('query=year', 'query=' + str(year)))
  
  return links

links_of_years = get_all_box()

def get_movie_link(link) :

  soup = get_soup(link)

  movie_list = []
  
  pannel = soup.select_one('.movie_audience_ranking._main_panel')
  minfo_list = pannel.select('._content div.movie_info')

  for minfo in minfo_list :
    a = minfo.select_one('a.movie_tit')['href']
    title = minfo.select_one('strong.scm_ellipsis_text').text
    mlink = link.split('&')[0] + a

    movie_list.append(mlink)

  return movie_list


mlink_list = get_movie_link(links_of_years[0])





In [437]:
# 영화 정보 스크랩 시작

def get_page(url, element=None) :
    
    if element == None :
        element = get_element(url)
        
    page = {
        'url' : url,
        'element' : element
    }
    
    return page

def get_element(url) :
    element = get_soup(url)
    
    return element

def get_movie_info(mlink) :
    movie_page = get_page(mlink)
    basis = get_basis(movie_page)
    director = get_director(movie_page)
    ratings = get_ratings(movie_page)
    
    return merge_dic(basis, director, ratings)

get_movie_info('https://search.naver.com/search.naver?where=nexearch&sm=tab_etc&pkid=68&os=1819142&qvt=0&query=%EC%8B%A0%EA%B3%BC%ED%95%A8%EA%BB%98-%EC%A3%84%EC%99%80%20%EB%B2%8C')

{'평점': '8.73', '참여수': '26,220명 참여', '남자평균': '8.71', '여자평균': '8.75'}
{'OST': '21', '영상미': '34', '스토리': '16', '연기': '8', '연출': '21'}
{'10대_평점': '9.09', '20대_평점': '8.76', '30대_평점': '8.66', '40대_평점': '8.73', '50대_평점': '8.44'}
{'10대_비율': '5%', '20대_비율': '45%', '30대_비율': '25%', '40대_비율': '19%', '50대_비율': '5%', 'male': '45%', 'female': '55%'}


{'개봉': '2017.12.20.',
 '등급': '12세 관람가',
 '장르': '판타지, 드라마',
 '국가': '대한민국',
 '러닝타임': '139분',
 '배급': '롯데엔터테인먼트',
 '원작': '웹툰',
 '감독': '김용화',
 '평점': '8.73',
 '참여수': '26,220명 참여',
 '남자평균': '8.71',
 '여자평균': '8.75',
 'OST': '21',
 '영상미': '34',
 '스토리': '16',
 '연기': '8',
 '연출': '21',
 '10대_평점': '9.09',
 '20대_평점': '8.76',
 '30대_평점': '8.66',
 '40대_평점': '8.73',
 '50대_평점': '8.44',
 '10대_비율': '5%',
 '20대_비율': '45%',
 '30대_비율': '25%',
 '40대_비율': '19%',
 '50대_비율': '5%',
 'male': '45%',
 'female': '55%'}

In [435]:
# 뷰숩으로 각 탭정보 가져오기

main_url = 'https://search.naver.com/search.naver'

def merge_url(suffix) :
    return main_url + suffix

def get_tab_page(movie_page, target) :
    tablist = movie_page['element'].select('.tab_list .tab')
    for tab in tablist :
        url = merge_url(tab.select_one('a')['href'])
        menu_name = tab.select_one('span.menu').text

        if menu_name == target :
            return get_page(url, get_soup(merge_url(tab.select_one('a')['href'])))

    return None

def get_basis(movie_page) :
    tab = get_tab_page(movie_page, '기본정보')
    details = tab['element'].select('._au_movie_content_wrap .detail_info .info_group')
    basic = {}

    for detail in details :
        key = detail.select_one('dt').text
        val = detail.select_one('dd').text
        basic[key] = val

    return basic


def get_director(movie_page) :
    tab = get_tab_page(movie_page, '감독/출연')  

    cast = tab['element'].select_one('.cast_box')
    name = cast.select_one('.cast_list strong.name span._text').text
    rst = {'감독' : name}
    
    return rst

def merge_dic(*args) :
    result = {}
    for dic in args :
        result.update(dic)
    
    return result

def get_ratings(movie_page) :
    rating_tab = get_tab_page(movie_page, '평점')

    star_info = get_star_info(rating_tab)
    point_info = get_point_info(rating_tab)
    age_info = get_age_info(rating_tab)
    tend_info = get_tend_info(rating_tab)
    
    raiting_rst = merge_dic(star_info, point_info, age_info, tend_info)
    
    return raiting_rst

In [ ]:
# 뷰숩으로 관람객 평점 정보 가져오기

def get_star_info(rating_tab) :
  see_box = rating_tab['element'].select_one('.lego_rating_box_see')
  total_rat = see_box.select_one('.area_intro_info .area_star_number').text
  population = see_box.select_one('.area_people').text
  male_rat = see_box.select_one('.area_card_male .area_star_number .this_text').text
  female_rat = see_box.select_one('.area_card_female .area_star_number .this_text').text

  star_info = {
      '평점' : total_rat,
      '참여수' : population,
      '남자평균' : male_rat,
      '여자평균' : female_rat
  }

  return star_info
  

In [402]:
# 셀레니움으로 감상 포인트 정보 가져오기

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
import time

def get_driver(url) :

    driver = webdriver.Chrome(service=service, options=options)
    driver.get(url)
    
    return driver


def get_point_info(page) :
    element = get_driver(page['url'])
    
    chart_con = element.find_element(By.CLASS_NAME, "_chart_container")
    time.sleep(1)
    cates = chart_con.find_element(By.CLASS_NAME, 'area_category').find_elements(By.TAG_NAME, 'li')
    
    
    cate_dic = {}
    for cate in cates :
        cate_dic[cate.text] = cate.text
        cid = cate.get_attribute('class').split('_')[1]
        add_cate_point(cid, cate.text, cate_dic)
    
    return cate_dic

def add_cate_point(cid, key, cate_dic) :
        label_class = 'this_label_' + cid
        label = chart_con.find_element(By.CLASS_NAME, 'area_label_box').find_element(By.CLASS_NAME, label_class)    

        cate_dic[key] = label.text

In [404]:
# 뷰숩으로 나이별 만족도 정보 가져오기

def get_age_info(rating_tab) :
    element = rating_tab['element']
    
    states = element.select('.lego_rating_box_age .area_graph_age li')
    rating_dic = {}
    
    for li in states :
        age = li.select_one('.area_text_age').text
        rating = li.select_one('.this_text_num').text
        rating_dic[age + '_평점'] = rating
        
    return rating_dic
    

In [406]:
# 관람 추이 가져오기
def get_tend_info(rating_tab) :
    element = get_driver(rating_tab['url'])
    time.sleep(1)
    menus = element.find_element(By.CLASS_NAME, '_cm_content_area_chart').find_elements(By.CLASS_NAME, 'tab')
    for menu in menus :
        if menu.find_element(By.CLASS_NAME, 'menu').text == '관람추이' :
            menu.find_element(By.TAG_NAME, 'a').click()
            
            male = element.find_element(By.CLASS_NAME, 'bb-target-남자')
            female = element.find_element(By.CLASS_NAME, 'bb-target-여자')

    gen_tend = {
        'male' : male.text,
        'female' : female.text
    }
            
    age = element.find_element(By.CLASS_NAME, 'lego_rating_box_change_age')
    age_texts = age.find_elements(By.CLASS_NAME, 'area_text_age')
    age_pers = age.find_elements(By.CLASS_NAME, 'area_text_percent')
    
    tend = {}
    for text, percent in zip(age_texts, age_pers) :
        key = text.text + '_비율'
        value = percent.text
        tend[key] = value,

    for key, value in tend.items() :
        tend[key] = ''.join(value)
        
    tend.update(gen_tend)
    return tend

In [422]:
get_movie_info('https://search.naver.com/search.naver?where=nexearch&sm=tab_etc&pkid=68&os=1819142&qvt=0&query=%EC%8B%A0%EA%B3%BC%ED%95%A8%EA%BB%98-%EC%A3%84%EC%99%80%20%EB%B2%8C')

KeyError: 'element'

In [52]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService

import time

options = webdriver.ChromeOptions()
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option("useAutomationExtension", False)
options.add_argument("--headless=new")
service = ChromeService(executable_path='C:/Users/Administrator/Downloads/google-driver/chromedriver.exe')
driver = webdriver.Chrome(service=service, options=options)
driver.get("http://www.python.org")


assert "Python" in driver.title

# find_element -find
# find_elements - find_all

# By.ID -> id 속성으로 찾기 => id속성값은 무조건 결과가 1개 아니면 0개
# By.CLASS_NAME -> class 속성으로 찾기
# By.TAG_NAME -> 태그 이름으로 찾기
# By.Name -> name 속성으로 찾기
elem = driver.find_element(By.NAME, "q")

elem2 = driver.find_element(By.CLASS_NAME, "donate-button")

# 텍스트 가져오기 - tag.text
print(elem2.text)

# 속성값 가져오기 - .get_attribute('속성명')
print(elem2.get_attribute('href'))



# assert "No results found." not in driver.page_source
# time.sleep(5)
# driver.close()

Donate
https://psfmember.org/civicrm/contribute/transact?reset=1&id=2


In [53]:
driver = webdriver.Chrome(service=service, options=options)
driver.get("https://www.naver.com")

# query = driver.find_element(By.ID, 'query')


# 입력 - tag.send_keys(입력값)
# query.send_keys("장마")
# query.send_keys(Keys.RETURN)
# time.sleep(10)

# 클릭 - tag.click()
navs = driver.find_elements(By.CLASS_NAME, 'nav_item')

target = None
for nav in navs :
    if nav.text == "카페" :
        target = nav
        
if target != None :
    target.click()

# 주의할 점 => 문서가 도착할 때까지 시간이 걸릴 수 있기 때문에 대기를 해줘야 함.
driver.implicitly_wait(10) 

input_text =driver.find_element(By.CLASS_NAME, 'input_text')
input_text.send_keys("고양이")
input_text.send_keys(Keys.RETURN)

In [34]:
area_graph_age

C:\Users\차태진\AppData\Local\Temp\ipykernel_2140\815279921.py:9: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('C:/Users/Administrator/Downloads/google-driver/chromedriver.exe')


{'번호': '0001204852', '댓글수': '658', '75%': '남자', '25%': '여자', '10대': '0%', '20대': '1%', '30대': '10%', '40대': '38%', '50대': '37%', '60대↑': '14%'}
